In [1]:
from fastai2.vision.all import *
from fastai2.vision.widgets import *
import chess
from utils import *

In [9]:
def predict_board(img):
    h,w = img.shape
    item_list = []
    for y in range(8):
        for x in range(8):
            box = (x*w/8, y*h/8, (x+1)*w/8, (y+1)*h/8 ) # left, top, right, bottom
            cropped = img.crop(box)
            square = PILImage.create(cropped.to_bytes_format())
            item_list.append(square) 

    dl = learner.dls.test_dl(item_list)
    _, __, preds = learner.get_preds(dl=dl, with_decoded=True)
    preds = [classes[pred] for pred in preds]
    labels = ''.join([label_lib[pred] for pred in preds])
    fen = label2fen(labels)
    
    return fen

In [10]:
learner = load_learner('lichess_finetuned.pkl')
classes = learner.dls.vocab

In [11]:
uploader = widgets.FileUpload()
out_pl = widgets.Output()
lbl_fen = widgets.Label()
btn_run = widgets.Button(description='Get position')

In [12]:
def on_click_classify(change):
    img = PILImage.create(uploader.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    fen = predict_board(img)
    lbl_fen.value = fen

btn_run.on_click(on_click_classify)

In [13]:
#hide_output
VBox([widgets.Label('Upload chess diagram'), 
      uploader, btn_run, lbl_fen])